In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv(r'C:\Users\Lenovo\Downloads\car.csv')
#df.shape

In [2]:
df["model"]=df["model"].values.astype('str')
df["origin"]=df["origin"].values.astype('str')
df["brand"]=df["brand"].values.astype('str')
df["gearbox"]=df["gearbox"].values.astype('str')
df["type"]=df["type"].values.astype('str')
df["fuel"]=df["fuel"].values.astype('str')
df["condition"]=df["condition"].values.astype('str')

df_new = df.drop_duplicates()

In [3]:
df_used = df_new[df_new['condition'] == 'used']
df_brand = df_used[df_used.brand.isin(['Toyota', 'Ford', 'Kia', 'Huyndai', 'Mitsubishi',
                                      'Mercedes Benz', 'Mazda', 'Honda', 'Chevrolet', 'Vinfast',
                                      'Suzuki', 'BMW', 'Nissan', 'Daewoo', 'Lexus', 'Peugeot',
                                      'Volkswagen', 'Audi', 'Isuzu', 'Porsche'])]
df_nonnull = df_brand.dropna(how = 'any', axis = 0)
df_date = df_nonnull[df_nonnull['manufacture_date'] > 0]
df_odo = df_date[df_nonnull['mileage_v2'] > 20]
df_final = df_date[df_date['condition'] == 'used']
df_end = df_final[df_final['type'].str.contains("nan") == False]
df_end = df_end[df_end['origin'].str.contains("nan") == False]
df_end = df_end[df_end['gearbox'].str.contains("nan") == False]
df_end = df_end[df_end['model'].str.contains("nan") == False]
df_end = df_end[df_end['model'].str.contains("Dòng khác") == False]
df_end.shape

(34771, 15)

In [4]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

df_end['brand'] = le.fit_transform(df_end['brand'])
df_end['origin'] = le.fit_transform(df_end['origin'])
df_end['type'] = le.fit_transform(df_end['type'])
df_end['gearbox'] = le.fit_transform(df_end['gearbox'])
df_end['fuel'] = le.fit_transform(df_end['fuel'])
df_end['model'] = le.fit_transform(df_end['model'])

temp = pd.get_dummies(df_end, columns = ['model'], dtype=int)
inp = temp.drop(['id', 'list_id', 'list_time', 'condition', 'color'], axis = 1)

X = inp.drop(['price'], axis = 1)
y = inp['price']

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.7, test_size = 0.3, random_state = 99)

#inp.head(10)

In [6]:
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import QuantileTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor

rfc = RandomForestRegressor(n_estimators= 1000, n_jobs = 7)

In [7]:
rfc.fit(X_train,y_train)

RandomForestRegressor(n_estimators=1000, n_jobs=7)

In [8]:
y_pred = rfc.predict(X_test)

In [9]:
from sklearn.metrics import mean_absolute_error
rate = mean_absolute_error(y_test, y_pred)
rate

33475806.691578027

In [10]:
control = (np.divide(y_test,y_pred) - 1)*100
tem = ((-5 < control) & (control < 5)).sum()
per = tem/(len(control))*100
per

73.39915644171779